In [1]:
import json
import numpy as np
from gensim.models import KeyedVectors
import random

In [2]:
model = KeyedVectors.load("wiktionary5800_custom.kv")

with open("korean_frequency.json", encoding="utf-8") as f:
    data = json.load(f)

In [3]:
word_list = sorted({w for group in data.values() for w in group if w in model})


In [4]:
def update_candidates(candidates, guess, similarity, tolerance=5.0):
    new_candidates = []
    for cand in candidates:
        if cand in model and guess in model:
            sim = model.similarity(cand, guess) * 100
            if abs(sim - similarity) <= tolerance:
                new_candidates.append(cand)
    return new_candidates

def select_best_guess(candidates, tried):
    max_entropy = -1
    best_word = None
    for word in candidates:
        if word not in model:
            continue
        sims = []
        for other in tried:
            if other in model:
                sims.append(model.similarity(word, other))
        entropy = -np.sum(np.log(np.clip(sims, 1e-5, 1))) if sims else 0
        if entropy > max_entropy:
            max_entropy = entropy
            best_word = word
    return best_word or random.choice(candidates)

In [5]:
def simulate_game(answer, verbose=True, max_steps=20):
    if answer not in model:
        raise ValueError(f"정답 '{answer}'이(가) 모델에 없습니다.")

    candidates = word_list.copy()
    tried = []

    for step in range(1, max_steps + 1):
        guess = select_best_guess(candidates, tried)
        tried.append(guess)

        if guess == answer:
            if verbose:
                print(f"✅ 정답 '{guess}'을 {step}번 만에 맞췄습니다!")
            return tried

        similarity = model.similarity(guess, answer) * 100
        if verbose:
            print(f"{step}. '{guess}' → 유사도: {similarity:.2f}")

        candidates = update_candidates(candidates, guess, similarity)

    if verbose:
        print(f"❌ 실패: {max_steps}회 이내에 정답을 못 맞힘")
    return tried

In [7]:
simulate_game("사랑스럽다", max_steps=100)

1. '가게' → 유사도: 77.55
2. '발걸음' → 유사도: 84.91
3. '꿈속' → 유사도: 78.08
4. '바깥' → 유사도: 79.91
5. '운동화' → 유사도: 81.25
6. '묶다' → 유사도: 86.08
7. '전문직' → 유사도: 86.77
8. '어머님' → 유사도: 85.95
9. '꾸미다' → 유사도: 84.91
10. '인형' → 유사도: 91.21
11. '소시지' → 유사도: 86.95
12. '과외' → 유사도: 88.55
13. '쓰이다' → 유사도: 86.47
14. '반갑다' → 유사도: 92.62
15. '되돌리다' → 유사도: 92.70
16. '바깥쪽' → 유사도: 93.08
17. '신세' → 유사도: 93.12
18. '쩔쩔매다' → 유사도: 93.48
19. '독특하다' → 유사도: 93.46
20. '양배추' → 유사도: 94.96
21. '부러지다' → 유사도: 94.63
✅ 정답 '사랑스럽다'을 22번 만에 맞췄습니다!


['가게',
 '발걸음',
 '꿈속',
 '바깥',
 '운동화',
 '묶다',
 '전문직',
 '어머님',
 '꾸미다',
 '인형',
 '소시지',
 '과외',
 '쓰이다',
 '반갑다',
 '되돌리다',
 '바깥쪽',
 '신세',
 '쩔쩔매다',
 '독특하다',
 '양배추',
 '부러지다',
 '사랑스럽다']